In [1]:
import csv
import numpy as np
import sklearn
import argparse
import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pylab as plt
import pandas as pd

In [2]:
    format = '%Y-%m-%d'

    dir = 'data/'
    holiday_file = open( dir + 'holidays_events.csv', 'r')
    oil_file = open( dir + 'oil.csv', 'r')
    store_file = open( dir + 'stores.csv', 'r')
    transact_file = open( dir + 'transactions.csv')
    train_file = open( dir + 'train.csv')
    test_file = open( dir + 'test.csv')
    
    oil_data = csv.DictReader(oil_file)
    oil_arr = [ [ row['date'], row['dcoilwtico']] for row in oil_data ]
    for i in range(len(oil_arr)):
        entry = oil_arr[i][1]
        date = datetime.datetime.strptime(oil_arr[i][0], format)
        entry = float(entry) if entry != '' else oil_arr[i-1][1] if i != 0 else float(oil_arr[i+1][1])
        oil_arr[i][1] = entry
    oil_dict = { row[0] : row[1] for row in oil_arr }
    
    transact_data = csv.DictReader(transact_file)
    transact_dict = {}
    for row in transact_data:
        nb = row['store_nbr']
        if nb not in transact_dict:
            transact_dict[nb] = []
        transact_dict[nb].append([datetime.datetime.strptime(row['date'], format), int(row['transactions'])])

In [3]:
    def make_predictions(data):
        df = pd.DataFrame(data, columns = ['date', 'price'])
        df.index = pd.to_datetime(df['date'])
        df.drop(columns = 'date', inplace=True)
        df.head()
        
        display(df)

        mod = SARIMAX(df, order = (1,1,1), seasonal_order = (1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
        results = mod.fit()

        predictions = results.forecast(16)
        dti = pd.date_range("2017-08-16", periods=16, freq="D")
        predictions.index = dti

        return predictions

In [15]:
transact_preds = {}
for entry, data in transact_dict.items():
    pred = make_predictions(data)
    transact_preds[entry] = pred
    display(pred)
    

,price
date,
2013-01-01,770
2013-01-02,1038
2013-01-03,887
2013-01-04,1054
2013-01-05,1355
...,...
2017-08-11,2195
2017-08-12,1946
2017-08-13,1107


/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.33337D+00    |proj g|=  1.28039D-01

At iterate    5    f=  7.24476D+00    |proj g|=  3.06791D-02

At iterate   10    f=  7.24412D+00    |proj g|=  6.37773D-03

At iterate   15    f=  7.23571D+00    |proj g|=  6.27946D-02

At iterate   20    f=  7.23467D+00    |proj g|=  1.89073D-03

At iterate   25    f=  7.23461D+00    |proj g|=  2.05207D-02

At iterate   30    f=  7.22945D+00    |proj g|=  1.17397D-01

At iterate   35    f=  7.19152D+00    |proj g|=  2.32872D-01

At iterate   40    f=  7.09190D+00    |proj g|=  6.55440D-02

At iterate   45    f=  7.09058D+00    |proj g|=  6.38347D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = nu

/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


2017-08-16    887.044468
2017-08-17    806.620369
2017-08-18    886.614394
2017-08-19    855.926926
2017-08-20    905.434300
2017-08-21    847.926959
2017-08-22    850.532413
2017-08-23    716.039315
2017-08-24    799.163751
2017-08-25    886.700186
2017-08-26    906.454664
2017-08-27    891.279849
2017-08-28    873.591295
2017-08-29    871.706621
2017-08-30    878.049184
2017-08-31    876.093573
Freq: D, Name: predicted_mean, dtype: float64

...

2017-08-16    768.998338
2017-08-17    761.671113
2017-08-18    751.816568
2017-08-19    753.946828
2017-08-20    760.455411
2017-08-21    764.747077
2017-08-22    763.310025
2017-08-23    754.384359
2017-08-24    764.843682
2017-08-25    751.634698
2017-08-26    753.203815
2017-08-27    753.497638
2017-08-28    760.873434
2017-08-29    759.909893
2017-08-30    752.447374
2017-08-31    754.005584
Freq: D, Name: predicted_mean, dtype: float64

,price
date,
2017-04-20,4068
2017-04-21,3151
2017-04-22,4046
2017-04-23,4209
2017-04-24,2682
...,...
2017-08-11,2957
2017-08-12,2804
2017-08-13,2433


/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.12726D+00    |proj g|=  1.63126D-01

At iterate    5    f=  6.01219D+00    |proj g|=  8.04925D-03

At iterate   10    f=  6.01063D+00    |proj g|=  4.63492D-03

At iterate   15    f=  6.01058D+00    |proj g|=  2.06585D-03

At iterate   20    f=  6.00821D+00    |proj g|=  3.26089D-02

At iterate   25    f=  5.93169D+00    |proj g|=  2.39780D-01

At iterate   30    f=  5.89310D+00    |proj g|=  1.02739D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

2017-08-16    2246.251411
2017-08-17    2209.895011
2017-08-18    2203.236472
2017-08-19    2162.404193
2017-08-20    2219.216461
2017-08-21    2214.596845
2017-08-22    2237.497450
2017-08-23    1946.409184
2017-08-24    1904.658307
2017-08-25    2036.717681
2017-08-26    2085.160616
2017-08-27    2133.662860
2017-08-28    2246.808558
2017-08-29    2337.983145
2017-08-30    2296.046358
2017-08-31    2101.453477
Freq: D, Name: predicted_mean, dtype: float64

In [4]:
train_reader = csv.DictReader(train_file)
train_data = [row for row in train_reader]

test_reader = csv.DictReader(test_file)
test_data = [row for row in test_reader]

In [5]:
sales_dict = {}
for row in train_data:
    nb = row['store_nbr']
    if nb not in sales_dict:
        sales_dict[nb] = {}
    dt = datetime.datetime.strptime(row['date'], format)
    if dt not in sales_dict[nb]:
        sales_dict[nb][dt] = []
    sales_dict[nb][dt].append(float(row['sales']))
sd_arr = {}
for entry in sales_dict:
    row = sales_dict[entry]
    vals = []
    for date in row:
        vals.append([date, sum(row[date]) / len(row[date])])
    sd_arr[entry] = vals

In [6]:
sd_cat = {}
for row in train_data:
    nb = row['family']
    if nb not in sd_cat:
        sd_cat[nb] = {}
    dt = datetime.datetime.strptime(row['date'], format)
    if dt not in sd_cat[nb]:
        sd_cat[nb][dt] = []
    sd_cat[nb][dt].append(float(row['sales']))
sdc_arr = {}
for entry in sd_cat:
    row = sd_cat[entry]
    vals = []
    for date in row:
        vals.append([date, sum(row[date]) / len(row[date])])
    sdc_arr[entry] = vals

In [16]:
print(sales_dict['1'])

{datetime.datetime(2013, 1, 1, 0, 0): 0.0}


In [7]:
sales_preds = {}
for entry, data in sd_arr.items():
    pred = make_predictions(data)
    sales_preds[entry] = pred
    display(pred)

,price
date,
2013-01-01,0.000000
2013-01-02,224.762061
2013-01-03,177.977091
2013-01-04,179.390273
2013-01-05,191.478334
...,...
2017-08-11,163.749758
2017-08-12,228.594636
2017-08-13,118.266121


/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.02390D+00    |proj g|=  2.66863D-01

At iterate    5    f=  5.94999D+00    |proj g|=  1.38124D-01

At iterate   10    f=  5.89350D+00    |proj g|=  4.88856D-01

At iterate   15    f=  5.88524D+00    |proj g|=  2.96113D-02

At iterate   20    f=  5.88491D+00    |proj g|=  1.64462D-02

At iterate   25    f=  5.88042D+00    |proj g|=  4.81670D-01

At iterate   30    f=  5.83831D+00    |proj g|=  1.47543D-01

At iterate   35    f=  5.73310D+00    |proj g|=  2.38177D-01

At iterate   40    f=  5.72948D+00    |proj g|=  5.94043D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg 

/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


2017-08-16    290.301537
2017-08-17    296.949174
2017-08-18    292.522022
2017-08-19    300.190196
2017-08-20    296.633709
2017-08-21    297.471757
2017-08-22    297.091143
2017-08-23    298.198161
2017-08-24    292.684229
2017-08-25    294.838393
2017-08-26    292.301132
2017-08-27    291.190540
2017-08-28    291.228754
2017-08-29    297.791675
2017-08-30    292.429099
2017-08-31    301.285884
Freq: D, Name: predicted_mean, dtype: float64

...

,price
date,
2013-01-01,0.000000
2013-01-02,480.832848
2013-01-03,316.112697
2013-01-04,245.004818
2013-01-05,395.670000
...,...
2017-08-11,471.810485
2017-08-12,512.166879
2017-08-13,549.357727


/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.66240D+00    |proj g|=  1.47166D-01

At iterate    5    f=  6.53534D+00    |proj g|=  4.62747D-02

At iterate   10    f=  6.50268D+00    |proj g|=  4.65164D-02

At iterate   15    f=  6.50101D+00    |proj g|=  2.68965D-03

At iterate   20    f=  6.50060D+00    |proj g|=  4.07750D-02

At iterate   25    f=  6.48673D+00    |proj g|=  1.31065D-01

At iterate   30    f=  6.45611D+00    |proj g|=  2.14618D-01

At iterate   35    f=  6.27739D+00    |proj g|=  1.29992D-01

At iterate   40    f=  6.26792D+00    |proj g|=  6.67529D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg 

/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


2017-08-16    562.683102
2017-08-17    537.204476
2017-08-18    540.532870
2017-08-19    565.977660
2017-08-20    574.515051
2017-08-21    577.595274
2017-08-22    590.575717
2017-08-23    598.326978
2017-08-24    577.634092
2017-08-25    575.911411
2017-08-26    581.008782
2017-08-27    572.740153
2017-08-28    562.332945
2017-08-29    560.416598
2017-08-30    578.077990
2017-08-31    577.697398
Freq: D, Name: predicted_mean, dtype: float64

In [8]:
spc = {}
for entry, data in sdc_arr.items():
    pred = make_predictions(data)
    spc[entry] = pred
    display(pred)

,price
date,
2013-01-01,0.000000
2013-01-02,4.722222
2013-01-03,2.981481
2013-01-04,3.129630
2013-01-05,6.333333
...,...
2017-08-11,8.166667
2017-08-12,7.462963
2017-08-13,8.907407


/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.31934D+00    |proj g|=  1.64103D-01

At iterate    5    f=  2.01110D+00    |proj g|=  1.60323D-01

At iterate   10    f=  1.87771D+00    |proj g|=  2.39968D-01

At iterate   15    f=  1.86816D+00    |proj g|=  5.22855D-03

At iterate   20    f=  1.86815D+00    |proj g|=  7.61264D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     20     33      1     0     0   7.613D-05   1.868D+00
  F =   1.8681513579251157     

CONVERG

/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


2017-08-16    6.698350
2017-08-17    6.478510
2017-08-18    6.424719
2017-08-19    7.728889
2017-08-20    7.559659
2017-08-21    7.731494
2017-08-22    7.810502
2017-08-23    7.106054
2017-08-24    7.244627
2017-08-25    6.910818
2017-08-26    7.910679
2017-08-27    7.874945
2017-08-28    7.542833
2017-08-29    7.495577
2017-08-30    7.668698
2017-08-31    7.481485
Freq: D, Name: predicted_mean, dtype: float64

...

,price
date,
2013-01-01,0.000000
2013-01-02,28.273148
2013-01-03,20.265018
2013-01-04,23.946685
2013-01-05,23.067352
...,...
2017-08-11,23.566963
2017-08-12,19.037593
2017-08-13,20.704574


/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.75858D+00    |proj g|=  3.78027D-01

At iterate    5    f=  3.13183D+00    |proj g|=  7.07685D-02

At iterate   10    f=  3.06883D+00    |proj g|=  2.53852D-01

At iterate   15    f=  3.00685D+00    |proj g|=  7.53435D-02

At iterate   20    f=  3.00509D+00    |proj g|=  1.25689D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     24     41      1     0     0   3.438D-05   3.005D+00
  F =   3.0050664146985966     

CONVERG

/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


2017-08-16    19.243601
2017-08-17    19.995083
2017-08-18    20.732518
2017-08-19    20.739829
2017-08-20    20.198798
2017-08-21    20.212818
2017-08-22    20.248594
2017-08-23    20.635874
2017-08-24    20.191933
2017-08-25    20.327398
2017-08-26    20.569580
2017-08-27    20.521751
2017-08-28    20.045528
2017-08-29    20.240137
2017-08-30    20.800379
2017-08-31    20.753771
Freq: D, Name: predicted_mean, dtype: float64

In [9]:
family_vals = {}
counter = 1
for row in train_data:
    fam = row['family']
    if fam not in family_vals:
        family_vals[fam] = counter
        counter += 1
        
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit([[row['family']] for row in train_data])

OneHotEncoder(handle_unknown='ignore')

In [25]:
enc_str = OneHotEncoder(handle_unknown='ignore')
enc_str.fit([[row['store_nbr']] for row in train_data])

OneHotEncoder(handle_unknown='ignore')

In [21]:
t = enc.transform([['HARDWARE']])
print(t.toarray()[0].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [26]:
train_ag = []
train_sales = []
td2 = {key : {row[0] : row[1] for row in val} for key,val in transact_dict.items()}
sd2 = {key : {row[0] : row[1] for row in val} for key,val in sd_arr.items()}
sdc2 = {key : {row[0] : row[1] for row in val} for key,val in sdc_arr.items()}
for row in train_data:
    rdate = row['date']
    rdtime = datetime.datetime.strptime(rdate, format)
    rstore = row['store_nbr']
    rcat = row['family']
    transact_val = td2[rstore][rdtime] if rdtime in td2[rstore] else 0
    sales_val = sd2[rstore][rdtime] if rdtime in sd2[rstore] else 0
    sc_val = sdc2[rcat][rdtime] if rdtime in sdc2[rcat] else 0
    oil_val = oil_dict[rdate] if rdate in oil_dict else 0
    fam_val = enc.transform([[row['family']]]).toarray()[0].tolist()
    store_val = enc_str.transform([[row['store_nbr']]]).toarray()[0].tolist()
#     entry = [rdtime.year, rdtime.month, rdtime.day, int(row['onpromotion']), oil_val, transact_val, sales_val, sc_val]
    entry = [rdtime.month, rdtime.day, int(row['onpromotion']), oil_val, transact_val, sales_val, sc_val]
    entry += fam_val
    entry += store_val
    train_ag.append(entry)
    train_sales.append(float(row['sales']))

In [27]:
test_ag = []
tp2 = {}
sp2 = {}
spc2 = {}
for store in transact_preds:
    p_dict = {}
    for ts, pred in transact_preds[store].items():
        dt = ts.to_pydatetime()
        dtstr = dt.strftime('2017-%m-%d')
        p_dict[dtstr] = pred
    tp2[store] = p_dict
for store in sales_preds:
    p_dict = {}
    for ts, pred in sales_preds[store].items():
        dt = ts.to_pydatetime()
        dtstr = dt.strftime('2017-%m-%d')
        p_dict[dtstr] = pred
    sp2[store] = p_dict
for store in spc:
    p_dict = {}
    for ts, pred in spc[store].items():
        dt = ts.to_pydatetime()
        dtstr = dt.strftime('2017-%m-%d')
        p_dict[dtstr] = pred
    spc2[store] = p_dict
for row in test_data:
    rdate = row['date']
    rdtime = datetime.datetime.strptime(rdate, format)
    rstore = row['store_nbr']
    rcat = row['family']
    transact_val = tp2[rstore][rdate] if rdate in tp2[rstore] else 0
    sales_val = sp2[rstore][rdate] if rdate in sp2[rstore] else 0
    sc_val = spc2[rcat][rdate] if rdate in spc2[rcat] else 0
    oil_val = oil_dict[rdate] if rdate in oil_dict else 0
    fam_val = enc.transform([[row['family']]]).toarray()[0].tolist()
    store_val = enc_str.transform([[row['store_nbr']]]).toarray()[0].tolist()
#     entry = [rdtime.year, rdtime.month, rdtime.day, int(row['onpromotion']), oil_val, transact_val, sales_val, sc_val]
    entry = [rdtime.month, rdtime.day, int(row['onpromotion']), oil_val, transact_val, sales_val, sc_val]
    entry += fam_val
    entry += store_val
    test_ag.append(entry)
    

In [28]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train_ag, train_sales)
predictions = reg.predict(test_ag)

In [29]:
with open('submission.csv', 'w', newline='') as ofile:
    w = csv.writer(ofile)
    w.writerow(['id', 'sales'])
    for i in range(len(test_data)):
        row = test_data[i]
        w.writerow([row['id'], predictions[i]])

In [33]:
print(sd2['1'])

{datetime.datetime(2013, 1, 1, 0, 0): 0.0, datetime.datetime(2013, 1, 2, 0, 0): 7417.147999999999, datetime.datetime(2013, 1, 3, 0, 0): 5873.244001, datetime.datetime(2013, 1, 4, 0, 0): 5919.879001, datetime.datetime(2013, 1, 5, 0, 0): 6318.78501, datetime.datetime(2013, 1, 6, 0, 0): 2199.0869999999995, datetime.datetime(2013, 1, 7, 0, 0): 6150.924, datetime.datetime(2013, 1, 8, 0, 0): 5597.181, datetime.datetime(2013, 1, 9, 0, 0): 6808.57899, datetime.datetime(2013, 1, 10, 0, 0): 4757.714999999999, datetime.datetime(2013, 1, 11, 0, 0): 5494.015979999999, datetime.datetime(2013, 1, 12, 0, 0): 5467.782980000001, datetime.datetime(2013, 1, 13, 0, 0): 2693.878004, datetime.datetime(2013, 1, 14, 0, 0): 5651.68501, datetime.datetime(2013, 1, 15, 0, 0): 5204.174, datetime.datetime(2013, 1, 16, 0, 0): 6733.596002, datetime.datetime(2013, 1, 17, 0, 0): 4942.473999999999, datetime.datetime(2013, 1, 18, 0, 0): 5923.124001, datetime.datetime(2013, 1, 19, 0, 0): 5895.146000000001, datetime.datetim

In [31]:
print(sdc2)

{'AUTOMOTIVE': {datetime.datetime(2013, 1, 1, 0, 0): 0.0, datetime.datetime(2013, 1, 2, 0, 0): 255.0, datetime.datetime(2013, 1, 3, 0, 0): 161.0, datetime.datetime(2013, 1, 4, 0, 0): 169.0, datetime.datetime(2013, 1, 5, 0, 0): 342.0, datetime.datetime(2013, 1, 6, 0, 0): 360.0, datetime.datetime(2013, 1, 7, 0, 0): 189.0, datetime.datetime(2013, 1, 8, 0, 0): 229.0, datetime.datetime(2013, 1, 9, 0, 0): 164.0, datetime.datetime(2013, 1, 10, 0, 0): 164.0, datetime.datetime(2013, 1, 11, 0, 0): 162.0, datetime.datetime(2013, 1, 12, 0, 0): 280.0, datetime.datetime(2013, 1, 13, 0, 0): 326.0, datetime.datetime(2013, 1, 14, 0, 0): 162.0, datetime.datetime(2013, 1, 15, 0, 0): 165.0, datetime.datetime(2013, 1, 16, 0, 0): 171.0, datetime.datetime(2013, 1, 17, 0, 0): 192.0, datetime.datetime(2013, 1, 18, 0, 0): 170.0, datetime.datetime(2013, 1, 19, 0, 0): 281.0, datetime.datetime(2013, 1, 20, 0, 0): 370.0, datetime.datetime(2013, 1, 21, 0, 0): 189.0, datetime.datetime(2013, 1, 22, 0, 0): 163.0, datet

In [32]:
print(spc2)

{'AUTOMOTIVE': {'2017-08-16': 362.1507435339138, '2017-08-17': 350.569901010617, '2017-08-18': 347.4496430468121, '2017-08-19': 417.3135797522455, '2017-08-20': 408.0316144732844, '2017-08-21': 417.25636712811325, '2017-08-22': 421.4152082111811, '2017-08-23': 383.96326295391, '2017-08-24': 391.30103659402994, '2017-08-25': 373.73508778859, '2017-08-26': 426.7592701276489, '2017-08-27': 424.8908125791958, '2017-08-28': 407.32812895473955, '2017-08-29': 404.73074132971976, '2017-08-30': 413.7258229577639, '2017-08-31': 404.15008984078935}, 'BABY CARE': {'2017-08-16': 9.065965895019074, '2017-08-17': 9.239526534853045, '2017-08-18': 9.316563780965222, '2017-08-19': 10.304353149223028, '2017-08-20': 9.119870165599007, '2017-08-21': 8.947789103312616, '2017-08-22': 8.855577160847833, '2017-08-23': 9.105348494107623, '2017-08-24': 9.077297441871679, '2017-08-25': 8.687156014034809, '2017-08-26': 9.084467641942975, '2017-08-27': 9.056239414726102, '2017-08-28': 9.25517862607584, '2017-08-29'

In [12]:
from sklearn.neural_network import MLPRegressor

mlp_reg = MLPRegressor(hidden_layer_sizes=(150,100,50),
                       max_iter = 300,activation = 'relu',
                       solver = 'adam')
mlp_reg.fit(train_ag, train_sales)
y_pred = mlp_reg.predict(test_ag)


/Users/ethanyu/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:
with open('submission.csv', 'w', newline='') as ofile:
    w = csv.writer(ofile)
    w.writerow(['id', 'sales'])
    for i in range(len(test_data)):
        row = test_data[i]
        w.writerow([row['id'], y_pred[i]])